### Intention of this notebook: 
This notebook is used to conduct data exploration, and find options to join the collision and building permit data sets. It first does some basic data cleaning, including changing data types where necessary and filtering out some unneeded data.

It then explores possibilities of joining the data using distance between building permits and collisions. 

In [212]:
import math
import timeit
import numpy as np
import pandas as pd
import re
from datetime import datetime 

collisions = pd.read_csv('Collisions.csv')
permits = pd.read_csv('Building_Permits___Current.csv')

# Clean data sets by changing data types (e.g., object to float or date type) and clarifying headers

#rename colunts X and Y to Longitude and Latitude for clarity
collisions = collisions.rename(index=str, columns={"X": "Longitude", "Y": "Latitude"})

#Turn date strings into datetime objects:
collisions['incdate'] = pd.to_datetime(collisions['incdate'], format='%Y-%m-%dT%H:%M:%S.%fZ')
permits['Issue Date']= pd.to_datetime(permits['Issue Date'], format='%m/%d/%Y')
permits['Final Date']= pd.to_datetime(permits['Final Date'], format='%m/%d/%Y')
permits['Expiration Date']= pd.to_datetime(permits['Expiration Date'], format='%m/%d/%Y')
permits['Application Date']= pd.to_datetime(permits['Application Date'], format='%m/%d/%Y')

# Change permit value from type Object to Type float, so we can choose permits above a specified value
permits['Value'] = permits['Value'].map(lambda x: x.lstrip('$'))
permits['Value'] =permits['Value'].str.replace(',', '').astype(float)



In [293]:
# Learn what permit types are. I think we are most interested in "New" construction
permit_types = set(permits['Action Type'])

# Filter permits to those we care most about. I choose new, and with cost of >$1M
new_permits = permits[permits['Action Type'] == 'NEW']
new_big_permits = new_permits[new_permits['Value'] > 1000000]

# Filter to include only permits with status issued or closed (this removes permits that haven't started building, or that never will)
new_big_permits_constructed = new_big_permits[new_big_permits['Status'].isin(['Permit Issued', 'Permit Closed'])] 

In [294]:
len(new_big_permits_constructed)

775

In [214]:
#This is the formula to calculate the distance, in miles betweent two lat/lon coordinates

def haversine(lat1, lon1, lat2, lon2):
    '''Calculates the distance in miles between two latitude/longitude coordinates.
    
    inputs: 
        lat1 = float, latitude of coordinate 1
        lon1 = float, longitude of coordinate 1
        lat2 = float, latitude of coordinate 2
        lon1 = float, longitude of coordinate 2
    
    returns: 
        float, distance betweet coordinate 1 and coordiante 2 in miles
    
    Note: 
    RADIUS is a global variable for the radius of the earth in miles, at the approximate latitude of the coordinates
'''
    #RADIUS is the radius of the earth at the latitude of Seattle
    RADIUS = 3955
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = RADIUS * c
    return total_miles

# Source: https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6

In [310]:
# Creating shorter data frames to use to test functions. Running the whole data frame takes a long time. 
short_collisions = collisions.head()
short_permits = new_big_permits.head(25)


In [145]:
# This function finds the closest permit to each collision

def find_closest_permit(collisions, buildings):
    '''Finds the distance of the nearest building permit to each collision
    
    inputs: 
        collisions: pandas.DataFrame with data on collisions
        buildings: pandas.DataFrame with data on building permits
    
    outputs: 
        :array of minimum distance to a building permit of each collision. Array 1 x length of collision data
    '''
    min_dists = []
    for index, row in collisions.iterrows():
        Lat = row['Latitude']
        Lon = row['Longitude']
        dists = haversine( Lat, Lon, buildings['Latitude'], buildings['Longitude'])
        min_dists.append(dists.min())
    return min_dists


In [162]:

distances = find_closest_permit(collisions, new_big_permits)
collisions['Dist to nearest permit'] = distances

In [286]:


def count_nearby_collisions(collisions, buildings, distance):
    ''' Counts how many collisions occured before, during and after the construction of each building permit
    
    inputs: 
        collisions: pandas dataframe with details of each collision, including location and date
        buildings: pandas dataframe with details of each building permit
        distance: float, radius from location of building permit for which user wishes to count collisions
        
    outputs:
        count_before: array, count of number of collisions that occured within specified radius before each building was constructed
        count_during: array, count of number of collisions that occured while specified building was under construction
        count_after: a count of number of collisions that occured within specified radius after building was constructed
        '''
    count_before = np.zeros(len(buildings))
    count_during = np.zeros(len(buildings))
    count_after = np.zeros(len(buildings))
    i = 0
    
    for index, row in buildings.iterrows():
#         print(row)
        Lat = row['Latitude']
        Lon = row['Longitude']
        dists = haversine( Lat, Lon, collisions['Latitude'], collisions['Longitude'])
        
        for j in range(0,len(dists)):
            if dists[j] > distance:
                continue
            else:
                if collisions.incdate[j] < row['Issue Date']:
                    count_before[i] += 1 
                elif (collisions.incdate[j] > row['Issue Date']) and collisions.incdate[j] < row['Final Date']:
                    count_during[i] += 1 
                elif (collisions.incdate[j] > row['Issue Date']) and (str(row['Final Date']) == 'NaT'):
                    count_during[i] += 1 
                elif collisions.incdate[j] > row['Final Date']:
                    count_after[i] += 1 
           
        i += 1
    return count_before, count_during, count_after


In [303]:
collisions_before, collisions_during, collisions_after = count_nearby_collisions(collisions, new_big_permits_constructed,.1)

In [307]:
# Append the count of collisions before, during and after the building was constructed to the permit data

new_big_permits_constructed['Collisions before'] = collisions_before
new_big_permits_constructed['Collisions during'] = collisions_during
new_big_permits_constructed['Collisions after'] = collisions_after


C:\Users\Alyssa\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Alyssa\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Alyssa\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [309]:
new_big_permits_constructed.to_csv('big_permits.csv')